In [1]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage
import plotly.express as px
import pandas as pd

# create clients
client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)


In [3]:
#### GET react, angular, vue.js QUESTIONS ##############

sample_size = 20000

query = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE REGEXP_CONTAINS(tags, r"vue.js") OR REGEXP_CONTAINS(tags, r"react") OR REGEXP_CONTAINS(tags, r"angular")
     OR REGEXP_CONTAINS(tags, r"svelte")  OR REGEXP_CONTAINS(tags, r"ember.js")  OR REGEXP_CONTAINS(tags, r"jquery")
    ORDER BY r
    LIMIT ''' + str(sample_size)


df = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

df

,r,tags,creation_date
0,5.611109e-07,angularjs|angular-promise,2014-09-04 16:52:46.773000+00:00
1,6.439181e-07,javascript|reactjs|vue.js|architecture,2020-09-01 20:03:31.093000+00:00
2,1.119253e-06,jquery,2013-03-18 10:35:23.560000+00:00
3,1.522181e-06,javascript|jquery|ajax|jsp,2015-03-21 18:25:26.177000+00:00
4,2.819044e-06,javascript|jquery|json|angularjs|frontend,2015-06-19 11:35:54.500000+00:00
...,...,...,...
19995,1.021704e-02,angular|frontend|observable|backend,2018-06-21 14:33:08.267000+00:00
19996,1.021762e-02,jquery|html|css,2013-10-27 18:26:06.710000+00:00
19997,1.021778e-02,javascript|android|reactjs|react-native,2020-07-06 15:36:16.493000+00:00
19998,1.021795e-02,jquery|filter|default,2011-06-01 16:16:19.287000+00:00


In [7]:
##### CHANGE TAG, SAVE TO CSV ################

def setTag(x):
    if(x.find('react') != -1):
        return 'react'
    elif(x.find('angular') != -1):
        return 'angular'
    elif(x.find('vue.js') != -1):
        return 'vue.js'
    elif(x.find('ember.js') != -1):
        return 'ember.js'
    elif(x.find('svelte') != -1):
        return 'svelte'
    elif(x.find('jquery') != -1):
        return 'jquery'
    

toSaveDf = pd.DataFrame(df)
toSaveDf = toSaveDf.drop(columns=['r'])
toSaveDf.rename(columns={'tags':'framework'}, inplace=True)
toSaveDf['framework'] = toSaveDf['framework'].map(setTag)
toSaveDf.to_csv('js-framework-tags-dataframe.csv')